In [1]:
!pip install vk

In [2]:
import json
import os
import sys
import time

In [3]:
from common.utils import *

In [4]:
module_path = os.path.abspath(os.path.join('.'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/xmanatee/Desktop/alyona/analytics


In [5]:
from settings import DATA_DIR

https://oauth.vk.com/authorize?client_id=6052784&scope=271502&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.63&response_type=token

In [6]:
# copy token here
token = "f77ac4525ee06ebcae3fdaa3e1b2ed3a936d1c91c5b89ad32d4af2ef3f26295a14426fe3a1ba9610b9b21"

In [7]:
poll_group_id = "-172053584"

# VK API

In [8]:
import vk
from vk.exceptions import VkAPIError

In [9]:
session = vk.Session(token)
api = vk.API(session, v="5.74")

# Getting wall data

### Creating data dir if it doesn't exist

In [10]:
os.makedirs(DATA_DIR, exist_ok=True)

### Requesting first 100

In [11]:
kwargs = {"owner_id": poll_group_id, "count": "100"}

group_wall_json = api.wall.get(**kwargs)

print("got {} items".format(len(group_wall_json["items"])))

got 100 items


### Requesting the rest

In [12]:
while len(group_wall_json["items"]) < group_wall_json["count"]:
    kwargs.update({
        "offset": len(group_wall_json["items"])
    })
    extra_group_wall_json = api.wall.get(**kwargs)
    group_wall_json["items"].extend(extra_group_wall_json["items"])
    print("Number of items now: {}".format(len(group_wall_json["items"])))
    time.sleep(3)

Number of items now: 200
Number of items now: 300
Number of items now: 400
Number of items now: 500
Number of items now: 600
Number of items now: 700
Number of items now: 800
Number of items now: 900
Number of items now: 1000
Number of items now: 1100
Number of items now: 1200
Number of items now: 1300
Number of items now: 1400
Number of items now: 1500
Number of items now: 1600
Number of items now: 1700
Number of items now: 1800
Number of items now: 1900
Number of items now: 2000
Number of items now: 2100
Number of items now: 2200
Number of items now: 2268


### Saving the result

In [13]:
with open(DATA_DIR + "group_wall.json", "w") as f:
    json.dump(group_wall_json, f)

# Getting poll voters

In [14]:
with open(DATA_DIR + "group_wall.json", "r") as f:
    group_wall_json = json.load(f)

In [15]:
def vote_in_poll(api, poll):
    answer_id = poll["answers"][0]["id"]
    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "answer_ids": [answer_id],
    }
    
    return api.polls.addVote(**kwargs)

In [16]:
for i, wall_post in enumerate(group_wall_json["items"]):
    poll = get_poll(wall_post)
    if poll is None or poll["anonymous"]:
        continue
    poll_filename = DATA_DIR + "poll_{}.json".format(poll["id"])

    if os.path.isfile(poll_filename):
        continue
    
    if poll["answer_id"] == 0:
        print("needs answering: https://vk.com/asurveys?w=poll-172053584_{}".format(poll["id"]))
        continue

    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "count": 1000,
        "answer_ids": list(map(lambda answer: answer["id"], poll["answers"])),
        "fields": "nickname, screen_name, sex, photo, photo_medium, photo_big"
    }
    
    try:
        poll_voters = api.polls.getVoters(**kwargs)
    except VkAPIError as vk_error:
        print(vk_error)
        print(poll)
        time.sleep(1)
        continue
    with open(poll_filename, "w") as f:
        json.dump(poll_voters, f)
    if (i + 1) % 20 == 0:
        print("Processed {}/{} polls".format(i + 1, len(group_wall_json["items"])))
    time.sleep(5)

needs answering: https://vk.com/asurveys?w=poll-172053584_346099976
needs answering: https://vk.com/asurveys?w=poll-172053584_345345136
needs answering: https://vk.com/asurveys?w=poll-172053584_336569489
needs answering: https://vk.com/asurveys?w=poll-172053584_306718174
